In [0]:
import numpy as np
import os
import pandas as pd
from sklearn.metrics import confusion_matrix
import h5py

**Load the dataset**

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [0]:
os.chdir('drive/My Drive/Colab Notebooks/DATA')

In [0]:
bogDF = pd.read_csv('blogtext.csv')

In [6]:
bogDF.head()

,id,gender,age,topic,sign,date,text
0,2059027,male,15,Student,Leo,"14,May,2004","Info has been found (+/- 100 pages,..."
1,2059027,male,15,Student,Leo,"13,May,2004",These are the team members: Drewe...
2,2059027,male,15,Student,Leo,"12,May,2004",In het kader van kernfusie op aarde...
3,2059027,male,15,Student,Leo,"12,May,2004",testing!!! testing!!!
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",Thanks to Yahoo!'s Toolbar I can ...


In [21]:
bogDF.shape

(681284, 7)

In [0]:
bogDFh=bogDF.head(10000)

In [236]:
bogDFh.shape

(10000, 7)

**Preprocess**

In [193]:
import nltk
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer,PorterStemmer
from nltk.corpus import stopwords
import re
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer() 
nltk.download('book_grammars')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package book_grammars to /root/nltk_data...
[nltk_data]   Package book_grammars is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [0]:
from datetime import datetime

now = datetime.now()


def preprocess(sentence):
    sentence=str(sentence)
    sentence = sentence.lower()
    sentence=sentence.replace('{html}',"") 
    cleanr = re.compile('<.*?>')
    cleantext = re.sub(cleanr, '', sentence)
    rem_url=re.sub(r'http\S+', '',cleantext)
    rem_num = re.sub('[0-9]+', '', rem_url)
    tokenizer = RegexpTokenizer(r'\w+')
    tokens = tokenizer.tokenize(rem_num)  
    filtered_words = [w for w in tokens if len(w) > 2 if not w in stopwords.words('english')]
    stem_words=[stemmer.stem(w) for w in filtered_words]
    lemma_words=[lemmatizer.lemmatize(w) for w in stem_words]
    return (" ".join(filtered_words))
def convertCommanToList(s):
    return s.split(",")

In [0]:
cleanTextDF=bogDFh['text'].map(lambda s:preprocess(s))

In [238]:
bogDFh.is_copy = False
bogDFh['cleanText']=pd.Series(list(cleanTextDF))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [0]:
bogDFh=bogDFh.drop(['text'],axis=1)

In [240]:
bogDFh.head()

,id,gender,age,topic,sign,date,cleanText
0,2059027,male,15,Student,Leo,"14,May,2004",info found pages pdf files wait untill team le...
1,2059027,male,15,Student,Leo,"13,May,2004",team members drewes van der laag urllink mail ...
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie aarde maak eigen water...
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture urls popups means...


In [241]:
bogDFh.dtypes

id            int64
gender       object
age           int64
topic        object
sign         object
date         object
cleanText    object
dtype: object

**Merge all the label columns together**

In [0]:
bogDFh['labels']= bogDFh['gender'].apply(str).str.cat(bogDFh['age'].apply(str), sep=",").str.cat(bogDFh['topic'].apply(str), sep=",").str.cat(bogDFh['sign'].apply(str), sep=",")

In [0]:
bogDFh['labels']= bogDFh['labels'].map(lambda s:convertCommanToList(s))

In [244]:
bogDFh.head()

,id,gender,age,topic,sign,date,cleanText,labels
0,2059027,male,15,Student,Leo,"14,May,2004",info found pages pdf files wait untill team le...,"[male, 15, Student, Leo]"
1,2059027,male,15,Student,Leo,"13,May,2004",team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]"
2,2059027,male,15,Student,Leo,"12,May,2004",het kader van kernfusie aarde maak eigen water...,"[male, 15, Student, Leo]"
3,2059027,male,15,Student,Leo,"12,May,2004",testing testing,"[male, 15, Student, Leo]"
4,3581210,male,33,InvestmentBanking,Aquarius,"11,June,2004",thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]"


In [0]:
bog_setDF=bogDFh.iloc[:,6:8]

**Separate features and labels, and split the data into training and testing**

In [0]:
from sklearn.model_selection import train_test_split


In [0]:
text_train, text_test, label_train, label_test = train_test_split(bog_setDF['cleanText'], bog_setDF['labels'], random_state=2)

**Vectorize the features**

In [0]:
from sklearn.feature_extraction.text import CountVectorizer
cvect = CountVectorizer(ngram_range=(1,2))

In [249]:

cvect.fit(text_train)

len(cvect.vocabulary_)

492924

In [0]:
Text_train_ct = cvect.transform(text_train)

In [251]:
Text_train_ct.shape
Text_train_ct[0]
print(Text_train_ct[0])

  (0, 397767)	1
  (0, 454804)	2
  (0, 456291)	1
  (0, 456478)	1


In [0]:
Text_test_ct = cvect.transform(text_test)


In [253]:
print(Text_test_ct[0])

  (0, 395)	1
  (0, 418)	1
  (0, 1820)	1
  (0, 3185)	1
  (0, 3543)	1
  (0, 7922)	1
  (0, 7934)	1
  (0, 11030)	1
  (0, 11283)	1
  (0, 11935)	1
  (0, 18387)	1
  (0, 25597)	1
  (0, 25614)	1
  (0, 26483)	1
  (0, 28937)	1
  (0, 32065)	1
  (0, 32501)	1
  (0, 32502)	1
  (0, 34938)	1
  (0, 51701)	1
  (0, 53781)	1
  (0, 55884)	1
  (0, 57425)	1
  (0, 57731)	1
  (0, 61281)	1
  :	:
  (0, 419063)	1
  (0, 428581)	1
  (0, 428829)	1
  (0, 429289)	1
  (0, 430361)	1
  (0, 434965)	2
  (0, 435629)	1
  (0, 435800)	1
  (0, 436451)	1
  (0, 441195)	1
  (0, 441829)	1
  (0, 442885)	2
  (0, 442987)	1
  (0, 453950)	1
  (0, 461100)	1
  (0, 469640)	1
  (0, 469940)	1
  (0, 469977)	1
  (0, 472266)	1
  (0, 472856)	1
  (0, 473483)	1
  (0, 473733)	1
  (0, 480596)	1
  (0, 481361)	1
  (0, 484372)	1


**Dictionary  count of every label**

In [0]:
 count_labels=dict(bogDFh['gender'].value_counts())

In [0]:
count_labels.update(dict(bogDFh['age'].value_counts()))
count_labels.update(dict(bogDFh['topic'].value_counts()))
count_labels.update(dict(bogDFh['sign'].value_counts()))

In [256]:
count_labels

{13: 42,
 14: 212,
 15: 602,
 16: 440,
 17: 1185,
 23: 253,
 24: 655,
 25: 386,
 26: 234,
 27: 1054,
 33: 136,
 34: 553,
 35: 2315,
 36: 1708,
 37: 33,
 38: 46,
 39: 79,
 40: 1,
 41: 20,
 42: 14,
 43: 6,
 44: 3,
 45: 16,
 46: 7,
 'Accounting': 4,
 'Aquarius': 571,
 'Aries': 4198,
 'Arts': 45,
 'Automotive': 14,
 'Banking': 16,
 'BusinessServices': 91,
 'Cancer': 504,
 'Capricorn': 215,
 'Communications-Media': 99,
 'Consulting': 21,
 'Education': 270,
 'Engineering': 127,
 'Fashion': 1622,
 'Gemini': 150,
 'HumanResources': 2,
 'Internet': 118,
 'InvestmentBanking': 70,
 'Law': 11,
 'LawEnforcement-Security': 10,
 'Leo': 301,
 'Libra': 491,
 'Marketing': 156,
 'Museums-Libraries': 17,
 'Non-Profit': 71,
 'Pisces': 454,
 'Publishing': 4,
 'Religion': 9,
 'Sagittarius': 1097,
 'Science': 63,
 'Scorpio': 971,
 'Sports-Recreation': 80,
 'Student': 1137,
 'Taurus': 812,
 'Technology': 2654,
 'Telecommunications': 2,
 'Virgo': 236,
 'female': 4084,
 'indUnk': 3287,
 'male': 5916}

**Transform the labels**

In [0]:
from sklearn.preprocessing import MultiLabelBinarizer

In [0]:
mlb = MultiLabelBinarizer()

In [0]:
 label_train_b=mlb.fit_transform(label_train)

In [261]:
mlb.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '41', '42', '43', '44', '45',
       '46', 'Accounting', 'Aquarius', 'Aries', 'Arts', 'Automotive',
       'Banking', 'BusinessServices', 'Cancer', 'Capricorn',
       'Communications-Media', 'Consulting', 'Education', 'Engineering',
       'Fashion', 'Gemini', 'HumanResources', 'Internet',
       'InvestmentBanking', 'Law', 'LawEnforcement-Security', 'Leo',
       'Libra', 'Marketing', 'Museums-Libraries', 'Non-Profit', 'Pisces',
       'Publishing', 'Religion', 'Sagittarius', 'Science', 'Scorpio',
       'Sports-Recreation', 'Student', 'Taurus', 'Technology',
       'Telecommunications', 'Virgo', 'female', 'indUnk', 'male'],
      dtype=object)

In [262]:
label_test_b=mlb.transform(label_test)

/usr/local/lib/python3.6/dist-packages/sklearn/preprocessing/_label.py:987: UserWarning: unknown class(es) ['40'] will be ignored
  .format(sorted(unknown, key=str)))


**Build model**

In [0]:
from sklearn.multiclass import OneVsRestClassifier
from sklearn.linear_model import LogisticRegression

In [264]:
clf = LogisticRegression(solver='lbfgs',max_iter=1000)
clf=OneVsRestClassifier(clf)
clf.fit(Text_train_ct, label_train_b)

OneVsRestClassifier(estimator=LogisticRegression(C=1.0, class_weight=None,
                                                 dual=False, fit_intercept=True,
                                                 intercept_scaling=1,
                                                 l1_ratio=None, max_iter=1000,
                                                 multi_class='auto',
                                                 n_jobs=None, penalty='l2',
                                                 random_state=None,
                                                 solver='lbfgs', tol=0.0001,
                                                 verbose=0, warm_start=False),
                    n_jobs=None)

In [226]:
    result = clf.classes_
    estimators = clf.estimators_
    print(result)


[ 0  1  2  3  4  5  6  7  8  9 10 11 12 13 14 15 16 17 18 19 20 21 22 23
 24 25 26 27 28 29 30 31 32 33 34 35 36 37 38 39 40 41 42 43 44 45 46 47
 48 49 50 51 52 53 54 55 56 57 58 59 60 61 62]


**Predictions and get the accuracy**

In [0]:
from sklearn import metrics

In [0]:
pred=clf.predict(Text_test_ct);

In [267]:
mlb.classes_

array(['13', '14', '15', '16', '17', '23', '24', '25', '26', '27', '33',
       '34', '35', '36', '37', '38', '39', '41', '42', '43', '44', '45',
       '46', 'Accounting', 'Aquarius', 'Aries', 'Arts', 'Automotive',
       'Banking', 'BusinessServices', 'Cancer', 'Capricorn',
       'Communications-Media', 'Consulting', 'Education', 'Engineering',
       'Fashion', 'Gemini', 'HumanResources', 'Internet',
       'InvestmentBanking', 'Law', 'LawEnforcement-Security', 'Leo',
       'Libra', 'Marketing', 'Museums-Libraries', 'Non-Profit', 'Pisces',
       'Publishing', 'Religion', 'Sagittarius', 'Science', 'Scorpio',
       'Sports-Recreation', 'Student', 'Taurus', 'Technology',
       'Telecommunications', 'Virgo', 'female', 'indUnk', 'male'],
      dtype=object)

In [268]:
metrics.accuracy_score(label_test_b.argmax(axis=1), pred.argmax(axis=1))

0.3896

In [0]:
matrix =confusion_matrix(label_test_b.argmax(axis=1), pred.argmax(axis=1))

In [274]:
matrix

array([[ 6,  0,  0, ...,  2,  0,  1],
       [ 0,  8,  2, ..., 10,  1,  8],
       [ 0,  0, 38, ..., 30,  4, 20],
       ...,
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0],
       [ 0,  0,  0, ...,  0,  0,  0]])

In [276]:
print(metrics.classification_report(label_test_b,pred))


              precision    recall  f1-score   support

           0       1.00      0.50      0.67        12
           1       0.73      0.13      0.22        61
           2       0.70      0.26      0.38       145
           3       0.71      0.21      0.32       121
           4       0.65      0.27      0.39       285
           5       0.25      0.02      0.03        66
           6       0.63      0.08      0.14       156
           7       0.67      0.09      0.16        86
           8       0.40      0.04      0.06        57
           9       0.73      0.41      0.53       253
          10       0.77      0.26      0.38        39
          11       0.94      0.72      0.82       137
          12       0.73      0.62      0.67       590
          13       0.96      0.54      0.69       427
          14       0.00      0.00      0.00         9
          15       0.75      0.23      0.35        13
          16       0.00      0.00      0.00        20
          17       0.00    

/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.6/dist-packages/sklearn/metrics/_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


**True label and predicted label for Top five**

In [0]:
 pred_labels=mlb.inverse_transform(pred)

In [0]:
bog_setDF['Predicted']= pd.Series(pred_labels)

In [286]:
bog_setDF.head(5)

,cleanText,labels,Predicted
0,info found pages pdf files wait untill team le...,"[male, 15, Student, Leo]","(male,)"
1,team members drewes van der laag urllink mail ...,"[male, 15, Student, Leo]","(Aries, male)"
2,het kader van kernfusie aarde maak eigen water...,"[male, 15, Student, Leo]","(35, Technology, male)"
3,testing testing,"[male, 15, Student, Leo]","(34, Sagittarius, female, indUnk)"
4,thanks yahoo toolbar capture urls popups means...,"[male, 33, InvestmentBanking, Aquarius]","(female, indUnk)"
